# Desafio 1

O desafio 1 é sobre a extração, limpeza e manipulação de dados no INEP sobre o Censo da Educação Superior. 

#### DADOS 

Estão disponíveis no site do Inep os Microdados do Censo da Educação Superior 2018 (DM_IES, DM_CURSO, DM_DOCENTE, DM_ALUNO, DM_LOCAL_OFERTA e TB_AUX_AREA_OCDE) em formato CSV delimitados por Pipe ( | ). Os arquivos encontram-se compactados (em formato .zip) pelo software 7-zip.  


#### ANEXO I – Dicionários de dados e Tabelas Auxiliares 

Contém, em formato .xlsx (Excel), o Dicionário de Dados do Censo da Educação Superior 2018 e também uma tabela auxiliar com o código e o nome dos países: 

1.DICIONÁRIO DE DADOS 

- TABELA DE ALUNO 
- TABELA DE CURSO 
- TABELA DE IES 
- TABELA DE LOCAL DE OFERTA 
- TABELA DE DOCENTE 
- TABELA AUXILIAR OCDE 
 
2.TABELA CONTENDO O NOME DO PAÍS DE ORIGEM OU NATURALIZAÇÃO 
 
#### ANEXO II – Questionários do Censo da Educação Superior  

Contém, em formato .pdf (Portable Document Format), os seguintes questionários do Censo da Educação Superior 2018 e estão disponíveis para download na pasta anexos: 

- MÓDULO IES 
- MÓDULO CURSO 
- MÓDULO DOCENTE 
- MÓDULO ALUNO 

### Leitura e preparação inicial dos dados

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import random
import glob
import scipy.stats as stats

In [2]:
#Lendo total de linhas de DM_ALUNO sem importar o dataset
with open ('dados/DM_ALUNO.csv') as f:
    total_linhas = sum(1 for row in f)-1
    
total_linhas

12043993

In [3]:
#Gerando um array de números aleatórios de 1 até total_linhas de tamanho = drop (99%)
percent_amostra = 0.01
amostra = int(percent_amostra * total_linhas)
drop = total_linhas - amostra

skip = random.sample(range(1,total_linhas),drop)

In [4]:
#Lendo o dataset com skiprows = 99% dos dados, gerando uma amostra de 1%. 
df_alunos = pd.read_csv('dados/DM_ALUNO.csv', sep='|', encoding='latin1', skiprows=skip)
df_alunos.shape

(120439, 105)

In [5]:
#Lendo os demais datasets
df_ies = pd.read_csv('dados/DM_IES.CSV', sep='|', encoding='latin1')
df_curso = pd.read_csv('dados/DM_CURSO.CSV', sep='|', encoding='latin1')
df_docente = pd.read_csv('dados/DM_DOCENTE.CSV', sep='|', encoding='latin1')
df_local = pd.read_csv('dados/DM_LOCAL_OFERTA.CSV', sep='|', encoding='latin1')
df_cine = pd.read_csv('dados/TB_AUX_CINE_BRASIL.CSV', sep='|', encoding='latin1')

##### 1) Crie duas tabelas: uma com número de universidades públicas e privadas por estado e outra com número de alunos por universidade 

Universidades Públicas e Privadas por estado:

In [6]:
#Usando a tabela do IBGE para atrelar código do estado a UF.
estados = pd.read_excel("dados/Tabela Estados IBGE.xlsx")
estados.head()

,Código da UF,UF,Estado
0,12,AC,Acre
1,27,AL,Alagoas
2,13,AM,Amazonas
3,16,AP,Amapá
4,29,BA,Bahia


In [7]:
estados = estados.rename(columns={'Código da UF':'CO_UF','UF':'SG_UF'}).drop('Estado',axis=1)
estados.head()

,CO_UF,SG_UF
0,12,AC
1,27,AL
2,13,AM
3,16,AP
4,29,BA


In [8]:
#Função que define universidade pública ou privada a partir de TP_CATEGORIA_ADMINISTRATIVA.
def publi_privada(categoria_adm):
    if categoria_adm < 3:
        return 'Universidade Pública'
    elif categoria_adm != 7:
        return 'Universidade Privada'

In [9]:
#Agrega estados ao df de uni / Aplica a função criando nova coluna como resposta / Agrupa por estado e tipo somando IES
df_publi_privada = df_ies.merge(estados)
df_publi_privada['TIPO_UNI'] = df_publi_privada['TP_CATEGORIA_ADMINISTRATIVA'].apply(publi_privada)
df_publi_privada.groupby(['SG_UF','TIPO_UNI']).agg(TOTAL_UNI = ('CO_IES','nunique'))

TOTAL_UNI
SG_UF TIPO_UNI                       
AC    Universidade Privada          9
      Universidade Pública          2
AL    Universidade Privada         25
      Universidade Pública          4
AM    Universidade Privada         20
      Universidade Pública          3
AP    Universidade Privada         12
      Universidade Pública          3
BA    Universidade Privada        136
      Universidade Pública         10
CE    Universidade Privada         76
      Universidade Pública          7
DF    Universidade Privada         62
      Universidade Pública          4
ES    Universidade Privada         74
      Universidade Pública          3
GO    Universidade Privada         94
      Universidade Pública          4
MA    Universidade Privada         49
      Universidade Pública          4
MG    Universidade Privada        281
      Universidade Pública         20
MS    Universidade Privada         29
      Universidade Pública          4
MT    Universidade Privada         58
      Universidade Pública          3
PA    Universidade Privada         51
      Universidade Pública          6
PB    Universidade Privada         41
      Universidade Pública          4
PE    Universidade Privada         90
      Universidade Pública          6
PI    Universidade Privada         42
      Universidade Pública          3
PR    Universidade Privada        185
      Universidade Pública         11
RJ    Universidade Privada        111
      Universidade Pública         25
RN    Universidade Privada         24
      Universidade Pública          5
RO    Universidade Privada         33
      Universidade Pública          2
RR    Universidade Privada          5
      Universidade Pública          3
RS    Universidade Privada        111
      Universidade Pública         10
SC    Universidade Privada         92
      Universidade Pública          5
SE    Universidade Privada         17
      Universidade Pública          2
SP    Universidade Privada        526
      Universidade Pública         82
TO    Universidade Privada         21
      Universidade Pública          3

Número de alunos por universidade:

In [10]:
#Agega o nome da universidade / Agrupa pelo nome da uni e conta o número de alunos ordenando de forma descendente
df_alunos_uni = df_alunos.merge(df_ies[['CO_IES','NO_IES']])
df_alunos_uni = df_alunos_uni.groupby('NO_IES', as_index=False).agg(
                                                N_ALUNOS = ('ID_ALUNO','nunique')).sort_values(by='N_ALUNOS',ascending=False)
df_alunos_uni.head()

,NO_IES,N_ALUNOS
2220,UNIVERSIDADE PAULISTA,6383
2221,UNIVERSIDADE PITÁGORAS UNOPAR,5795
2151,UNIVERSIDADE ESTÁCIO DE SÁ,4322
2065,UNIVERSIDADE ANHANGUERA,3286
136,CENTRO UNIVERSITÁRIO INTERNACIONAL,3057


In [11]:
#Como é uma amostra de 1% do total, pode-se multiplicar por 1/percent_amostra para se aproximar do número total de alunos
df_alunos_uni['N_ALUNOS'] = (df_alunos_uni['N_ALUNOS'] * 1/percent_amostra).astype(int)
df_alunos_uni

,NO_IES,N_ALUNOS
2220,UNIVERSIDADE PAULISTA,638300
2221,UNIVERSIDADE PITÁGORAS UNOPAR,579500
2151,UNIVERSIDADE ESTÁCIO DE SÁ,432200
2065,UNIVERSIDADE ANHANGUERA,328600
136,CENTRO UNIVERSITÁRIO INTERNACIONAL,305700
...,...,...
1078,FACULDADE FACMIL,100
1079,FACULDADE FACTUM,100
1081,FACULDADE FADAM DE MARACANAÚ,100
1084,FACULDADE FIA DE ADMINISTRAÇÃO E NEGÓCIOS,100


##### 2) Pergunta-se: é verdade que existem menos mulheres nos cursos de exatas? Explique com os dados.

In [12]:
display(df_cine[['CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL']].drop_duplicates())

,CO_CINE_AREA_GERAL,NO_CINE_AREA_GERAL
0,0,Programas básicos
10,1,Educação
67,2,Artes e humanidades
120,3,"Ciências sociais, jornalismo e informação"
137,4,"Negócios, administração e direito"
165,5,"Ciências naturais, matemática e estatística"
187,6,Computação e Tecnologias da Informação e Comun...
200,7,"Engenharia, produção e construção"
297,8,"Agricultura, silvicultura, pesca e veterinária"
318,9,Saúde e bem-estar


In [13]:
#Nota-se que 5, 6 e 7 são cursos de exatas.
exatas = [5,6,7]

In [14]:
#Agrega o nome e código da área ao df_alunos e atribui a df_exatas
df_exatas = df_alunos.merge(df_cine[['CO_CINE_ROTULO', 'CO_CINE_AREA_GERAL', 'NO_CINE_AREA_GERAL']])

In [15]:
#Mantém em df_exatas somente os alunos que o cód da área esteja em exatas.
df_exatas = df_exatas.loc[df_exatas['CO_CINE_AREA_GERAL'].isin(exatas),['ID_ALUNO','TP_SEXO','NO_CINE_AREA_GERAL']]
df_exatas.head()

,ID_ALUNO,TP_SEXO,NO_CINE_AREA_GERAL
12315,A37147F7FD0313BF1F9C17AD3A401FD0,2,"Engenharia, produção e construção"
12316,A030FF7DCA87821BB4A831265B999A72,2,"Engenharia, produção e construção"
12317,2FD1EC5160A214B09106F59B07F914F6,2,"Engenharia, produção e construção"
12318,F69016D70ACD8516159F2A6E6DEDE0D0,2,"Engenharia, produção e construção"
12319,CC2779E8AD8FB5AFA11208B49734028A,2,"Engenharia, produção e construção"


In [16]:
#Para garantir que df_exatas não tem alunos duplicados. 
df_exatas = df_exatas.drop_duplicates(subset='ID_ALUNO')
df_exatas.head()

,ID_ALUNO,TP_SEXO,NO_CINE_AREA_GERAL
12315,A37147F7FD0313BF1F9C17AD3A401FD0,2,"Engenharia, produção e construção"
12316,A030FF7DCA87821BB4A831265B999A72,2,"Engenharia, produção e construção"
12317,2FD1EC5160A214B09106F59B07F914F6,2,"Engenharia, produção e construção"
12318,F69016D70ACD8516159F2A6E6DEDE0D0,2,"Engenharia, produção e construção"
12319,CC2779E8AD8FB5AFA11208B49734028A,2,"Engenharia, produção e construção"


In [17]:
print(f'Total de mulheres nos cursos de exatas',df_exatas.loc[df_exatas['TP_SEXO']==1,'ID_ALUNO'].count())
print(f'Total de homens nos cursos de exatas',df_exatas.loc[df_exatas['TP_SEXO']==2,'ID_ALUNO'].count())

Total de mulheres nos cursos de exatas 6619
Total de homens nos cursos de exatas 16423


De fato existem menos mulheres do que homens nos cursos de exatas.

##### 3) Quantos cursos novos abrem por ano? 

In [18]:
df_novos_cursos = df_curso.dropna(subset=['DT_INICIO_FUNCIONAMENTO']).copy()
df_novos_cursos['DT_INICIO_FUNCIONAMENTO'].isnull().sum()

0

In [19]:
#df_novos_cursos['DT_INICIO_FUNCIONAMENTO'] = pd.to_datetime(df_novos_cursos['DT_INICIO_FUNCIONAMENTO'],format='%d/%m/%Y')
#Ao tentar converter a coluna para data, nota-se que há um erro de digitação em uma das linhas. 
df_novos_cursos['DT_INICIO_FUNCIONAMENTO'] = df_novos_cursos['DT_INICIO_FUNCIONAMENTO'].str.replace('2917','2017')

In [20]:
#Agora sim, convertendo a coluna e criando uma nova apenas com o ano de inicio.
df_novos_cursos['DT_INICIO_FUNCIONAMENTO'] = pd.to_datetime(df_novos_cursos['DT_INICIO_FUNCIONAMENTO'],format='%d/%m/%Y')
df_novos_cursos['ANO_INICIO_FUNCIONAMENTO'] = df_novos_cursos['DT_INICIO_FUNCIONAMENTO'].dt.year

In [21]:
#Agrupando pelo ano e inicio e contando os cursos
df_novos_cursos = (df_novos_cursos.groupby('ANO_INICIO_FUNCIONAMENTO', as_index=False).
                       agg(N_NOVOS_CURSOS = ('CO_CURSO','count')).sort_values(by='ANO_INICIO_FUNCIONAMENTO', ascending=False))

In [22]:
#Considerando um periodo de 10 anos.
anos = 10
ult = df_novos_cursos['ANO_INICIO_FUNCIONAMENTO'].max()
df_novos_cursos = df_novos_cursos.loc[df_novos_cursos['ANO_INICIO_FUNCIONAMENTO'] >= ult - anos]
df_novos_cursos

,ANO_INICIO_FUNCIONAMENTO,N_NOVOS_CURSOS
134,2018,1249
133,2017,1459
132,2016,1827
131,2015,1975
130,2014,1831
129,2013,1415
128,2012,1425
127,2011,1583
126,2010,1859
125,2009,2172


In [23]:
print(f'Abrem em média {int(df_novos_cursos["N_NOVOS_CURSOS"].mean())} novos cursos por ano')

Abrem em média 1669 novos cursos por ano


##### 4) A afirmação a seguir é verdadeira: Alunos da região Norte têm maior tendência a não concluírem os cursos quando comparados ao restante do país? Comprove com dados.

Considerando desistente os alunos que trancaram a matrícula ou estão desvinculados do curso (códigos 3 e 4).

In [49]:
#Agrega código de UF e IES ao df_alunos em df_desistente / Desistente = 1 - Não desistente = 0 /Remove alunos duplicados
df_desistente = df_alunos.merge(df_ies[['CO_IES','CO_UF']])
df_desistente['DESISTENTE'] = np.where((df_desistente['TP_SITUACAO'] == 3) | (df_desistente['TP_SITUACAO'] == 4),1,0)
df_desistente = df_desistente.drop_duplicates(subset=['ID_ALUNO'])

Estados da Região Norte e respecitvos CO_UF:
- Acre: 12
- Amapá: 16
- Amazonas: 13
- Pará: 15
- Rondônia: 11
- Roraima: 14
- Tocantins: 17

In [25]:
#Separando em dois dataframes alunos da região norte e demais regiões.
norte = [11,12,13,14,15,16,17]
df_alunos_norte = df_desistente.loc[df_desistente['CO_UF'].isin(norte)]
df_alunos_outras = df_desistente.loc[~df_desistente['CO_UF'].isin(norte)]
df_alunos_norte.shape, df_alunos_outras.shape

((6264, 107), (114038, 107))

In [26]:
print('O indice de desistência da região Norte é de {:.2f} %'.
      format((df_alunos_norte.loc[df_alunos_norte['DESISTENTE']==1, 'DESISTENTE'].count() * 100) / df_alunos_norte.shape[0]))
print('O indice de desistência de outras regiões é de {:.2f} %'.
      format((df_alunos_outras.loc[df_alunos_outras['DESISTENTE']==1, 'DESISTENTE'].count() * 100) / df_alunos_outras.shape[0]))

O indice de desistência da região Norte é de 25.37 %
O indice de desistência de outras regiões é de 29.11 %


A afirmação é falsa.

###### 5) Crie uma variável que represente a taxa de abandono para cada IES. É correto afirmar professores mais/menos capacitados influenciam tal taxa?

In [50]:
#Removendo professores duplicados / Agrupando por IES / Tirando a média de escolaridade dos professores
df_prof = df_docente.drop_duplicates(subset='ID_DOCENTE')
df_prof = df_prof.groupby('CO_IES', as_index = False).agg(ESCOLARIDADE = ('TP_ESCOLARIDADE','mean'))

In [51]:
#Agrupando alunos desistentes por IES / Calculando Taxa de abondono 
df_desistente = df_desistente.groupby('CO_IES', as_index=False).agg(TAXA_ABANDONO = ('DESISTENTE','mean'))

In [52]:
# Juntando os os dois dataframes
df_aluno_docente = df_prof.merge(df_desistente)

In [53]:
#Calculando a correlação entre a escolaridade do professor e a taxa de abandono. 
pearson, p_pearson = stats.pearsonr(df_aluno_docente['ESCOLARIDADE'], df_aluno_docente['TAXA_ABANDONO'])
spearman, p_spearman = stats.spearmanr(df_aluno_docente['ESCOLARIDADE'], df_aluno_docente['TAXA_ABANDONO'])

print(f'Correlação de Pearson: {round(pearson, 2)}, com p-valor de {round(p_pearson, 3)}')
print(f'Correlação de Spearman: {round(spearman, 2)}, com p-valor de {round(p_spearman, 3)}')

Correlação de Pearson: -0.09, com p-valor de 0.0
Correlação de Spearman: -0.05, com p-valor de 0.032


- Como o p-valor da correlação de Pearson e de Spearman é menor do que 0.05 há um fraca correlação.
- A correlação negativa nos diz que quanto maior a escolaridade, menor a taxa de abandono nas IES.

##### 6) Quais os cursos com maior crescimento de matriculas por região? E quais os com maior queda? Como você explicaria isso.

In [54]:
# Criando mapa de estados por região
regiao = {'Norte' : [11, 12, 13, 14, 15, 16, 17],
          'Nordeste' : [21, 22, 23 ,24, 25, 26, 27, 28, 29],
          'Centro-Oeste' : [50, 51, 52, 53],
          'Sudeste' : [31, 32, 33, 35],
          'Sul' : [41, 42, 43]}

In [71]:
#Trazendo as colunas com código e nome do curso e código do estado
df_matriculas = df_alunos.merge(df_curso[['CO_CURSO','NO_CURSO','CO_UF']])

In [72]:
#Função para classificar a região do curso a partir do CO_UF
def reg_estado(co_uf, regiao):
    for reg in regiao.keys():
        if co_uf in regiao[reg]:
            return reg
        
df_matriculas['REGIAO'] = df_matriculas['CO_UF'].apply(reg_estado, args=(regiao,))

In [73]:
#Agrupando por região, curso e ano. / Calculando o número de mátriculas por região, curso e ano. 
df_matriculas = df_matriculas.groupby(['REGIAO','NO_CURSO','NU_ANO_INGRESSO'], as_index=False).agg\
                                                                                        (N_MATRICULAS = ('ID_ALUNO','nunique'))
df_matriculas['N_MATRICULAS'] = df_matriculas['N_MATRICULAS'] / percent_amostra

In [74]:
#Filtrando os anos de 2017 e 2018
df_matriculas = df_matriculas.loc[df_matriculas['NU_ANO_INGRESSO'].isin([2018,2017])]

In [75]:
#Criando colunas separadas com matrículas de 2017 e 2018 / Renomeando colunas / Dropando valores nulos
df_matriculas = df_matriculas.pivot(index=['REGIAO','NO_CURSO'], columns=['NU_ANO_INGRESSO'],values=['N_MATRICULAS'])
df_matriculas = df_matriculas.reset_index()
df_matriculas.columns=['REGIAO','NO_CURSO','MATRICULAS_2017','MATRICULAS_2018']
df_matriculas.dropna(inplace=True)

In [77]:
#Calculando a difereça de uma ano pra outro
df_matriculas['AUMENTO_MATRICULAS'] = df_matriculas['MATRICULAS_2018'] - df_matriculas['MATRICULAS_2017']
df_matriculas

,REGIAO,NO_CURSO,MATRICULAS_2017,MATRICULAS_2018,AUMENTO_MATRICULAS
1,Centro-Oeste,ABI - CIÊNCIA DA COMPUTAÇÃO,100.0,100.0,0.0
2,Centro-Oeste,ABI - CIÊNCIAS SOCIAIS,300.0,200.0,-100.0
3,Centro-Oeste,ABI - COMUNICAÇÃO SOCIAL,100.0,100.0,0.0
6,Centro-Oeste,ABI - ENGENHARIA,600.0,600.0,0.0
8,Centro-Oeste,ABI - GEOGRAFIA,100.0,100.0,0.0
...,...,...,...,...,...
1348,Sul,TEOLOGIA,800.0,800.0,0.0
1349,Sul,TERAPIA OCUPACIONAL,300.0,300.0,0.0
1351,Sul,TURISMO,300.0,500.0,200.0
1353,Sul,TURISMO E MEIO AMBIENTE,100.0,100.0,0.0


In [86]:
#Criando dataframes para maiores e menores matriculas por região. 
df_maiores = pd.DataFrame(columns=df_matriculas.columns)
df_menores = pd.DataFrame(columns=df_matriculas.columns)

In [87]:
#Append nos 5 maiores cursos de cada região
for regiao in df_matriculas['REGIAO'].unique():
    aux = df_matriculas.loc[df_matriculas['REGIAO'] == regiao]
    df_maiores = df_maiores.append(aux.nlargest(5, 'AUMENTO_MATRICULAS'))
    df_menores = df_menores.append(aux.nsmallest(5, 'AUMENTO_MATRICULAS'))

In [88]:
df_maiores

,REGIAO,NO_CURSO,MATRICULAS_2017,MATRICULAS_2018,AUMENTO_MATRICULAS
21,Centro-Oeste,AGRONOMIA,3300.0,5800.0,2500.0
67,Centro-Oeste,DIREITO,30100.0,32500.0,2400.0
161,Centro-Oeste,NUTRIÇÃO,3400.0,5500.0,2100.0
73,Centro-Oeste,ENFERMAGEM,10100.0,12000.0,1900.0
102,Centro-Oeste,FISIOTERAPIA,5600.0,7100.0,1500.0
277,Nordeste,DIREITO,47300.0,55200.0,7900.0
335,Nordeste,FARMÁCIA,7500.0,13300.0,5800.0
210,Nordeste,ADMINISTRAÇÃO,24500.0,28400.0,3900.0
431,Nordeste,PEDAGOGIA,19400.0,23100.0,3700.0
242,Nordeste,CIÊNCIAS CONTÁBEIS,16900.0,19000.0,2100.0


In [85]:
df_menores

,REGIAO,NO_CURSO,MATRICULAS_2017,MATRICULAS_2018,AUMENTO_MATRICULAS
71,Centro-Oeste,EDUCAÇÃO FÍSICA,7600.0,6500.0,-1100.0
92,Centro-Oeste,ENGENHARIA ELÉTRICA,1700.0,700.0,-1000.0
195,Centro-Oeste,ZOOTECNIA,1500.0,600.0,-900.0
153,Centro-Oeste,MARKETING,700.0,100.0,-600.0
108,Centro-Oeste,GEOGRAFIA,1500.0,1000.0,-500.0
338,Nordeste,FISIOTERAPIA,16200.0,14700.0,-1500.0
427,Nordeste,NUTRIÇÃO,11700.0,10300.0,-1400.0
428,Nordeste,ODONTOLOGIA,9600.0,8200.0,-1400.0
221,Nordeste,ARQUITETURA E URBANISMO,7200.0,6000.0,-1200.0
458,Nordeste,SERVIÇO SOCIAL,6800.0,5600.0,-1200.0
